# Overview
- とりあえずデータ見る

# Const

In [77]:
NB = '002'
PATH_TRAIN = './../data/official/train.csv'
PATH_TEST = './../data/official/test.csv'
PATH_SAMPLE_SUBMITTION = './../data/official/atmaCup8_sample-submission.csv'
SAVE_DIR = f'../data/output_nb/nb{NB}/'

# Import everything I need :)

In [82]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor 
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

# My function

In [14]:
def metric(y_true, y_pred):
    return mean_squared_log_error(y_true, y_pred) ** .5

# Preparation

set

In [83]:
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

<br>

load dataset

In [84]:
train = pd.read_csv(PATH_TRAIN)
test = pd.read_csv(PATH_TEST)
ss = pd.read_csv(PATH_SAMPLE_SUBMITTION)

# EDA

In [85]:
train.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,LEGO Batman: The Videogame,Wii,NaN,Action,Warner Bros. Interactive Entertainment,180,97,0,28,306,74.0,17.0,7.9,22.0,Traveller's Tales,E10+
1,LEGO Indiana Jones: The Original Adventures,Wii,NaN,Action,LucasArts,151,61,0,21,234,78.0,22.0,6.6,28.0,Traveller's Tales,E10+
2,LEGO Batman: The Videogame,PSP,NaN,Action,Warner Bros. Interactive Entertainment,56,44,0,27,128,73.0,5.0,7.4,10.0,Traveller's Tales,E10+
3,Combat,2600,NaN,Action,Atari,117,7,0,1,125,NaN,NaN,NaN,NaN,NaN,NaN
4,LEGO Harry Potter: Years 5-7,Wii,NaN,Action,Warner Bros. Interactive Entertainment,69,42,0,12,124,76.0,8.0,7.8,13.0,Traveller's Tales,E10+


In [86]:
test.head()

,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Hitman 2: Silent Assassin,XB,NaN,Action,Eidos Interactive,84.0,23.0,8,19.0,Io Interactive,M
1,Legacy of Kain: Soul Reaver,PS,NaN,Action,Eidos Interactive,91.0,17.0,9,132.0,Crystal Dynamics,T
2,Metal Gear Solid 2: Substance,XB,NaN,Action,Konami Digital Entertainment,87.0,28.0,8.5,39.0,KCEJ,M
3,Silent Hill: Homecoming,X360,NaN,Action,Konami Digital Entertainment,70.0,54.0,6.9,180.0,Double Helix Games,M
4,Silent Hill: Homecoming,PS3,NaN,Action,Konami Digital Entertainment,71.0,41.0,6.9,143.0,Double Helix Games,M


In [87]:
test.columns

Index(['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher',
       'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Developer',
       'Rating'],
      dtype='object')

In [88]:
use_feat = ['Critic_Score', 'Critic_Count', 'User_Count']

In [89]:
X = train[use_feat].copy()
y = train[['Global_Sales']].copy()
X_te = test[use_feat].copy()

# Create Model

In [90]:
model = LGBMRegressor(random_state=2020)

In [91]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)
model.fit(X_train.values, y_train.values[:, 0])

y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_valid)

<br>

post processing

In [92]:
mask = y_train_pred <= 1
y_train_pred[mask] = 1

mask = y_valid_pred <= 1
y_valid_pred[mask] = 1

In [93]:
print(f'score train: {metric(y_train, y_train_pred):.5f}')
print(f'score valid: {metric(y_valid, y_valid_pred):.5f}')

score train: 1.56401
score valid: 1.64816


# create sub

In [94]:
y_test_pred = model.predict(X_te)
mask = y_test_pred <= 1
y_test_pred[mask] = 1

ss['Global_Sales'] = y_test_pred

In [97]:
save_path = f'{SAVE_DIR}submission.csv'
ss.to_csv(save_path, index=False)

print(f'save: {save_path}')

save: ../data/output_nb/nb002/submission.csv
